# A callable is contravariant in its arguments

In december 2024 I became interested in "PEP 483 – The Theory of Type Hints" and particulary Covariance and Contravariance as described in https://peps.python.org/pep-0483/#covariance-and-contravariance.

The subject of covariance and contravariance in programming is not an easy topic. Here is what I understand of this topic, at least for a callable.

As defined in PEP 483 :

If t2 is a subtype of t1, then a generic type constructor GenType is called:

Covariant, if GenType[t2] is a subtype of GenType[t1] for all such t1 and t2.
Contravariant, if GenType[t1] is a subtype of GenType[t2] for all such t1 and t2.
Invariant, if neither of the above is true.
Later on, it is said that "a callable is covariant in the return type, but contravariant in the arguments" so :

Callable[[], Manager] is a subtype of Callable[[], Employee]
Callable[[Employee], None] is a subtype of Callable[[Manager], None]
It seems natural that a callable would be covarient in the return type but why contravariant in the arguments ? An example is given in the PEP 483 but I struggled to understand well this concept so I made my own program based on the example given in the PEP to understand better.

Lets suppose we have a companie with managers and employees. The salary of an employee (not manager) is just his base_salary and for a manager it is (for example) base_salary + 100 * number_of_subordonates :

In [1]:

from collections.abc import Callable
from functools import partial


class Employee:
    def __init__(self, name: str, base_salary: float):
        self.name = name
        self.base_salary = base_salary


class Manager(Employee):
    def __init__(self, name: str, base_salary: float, number_of_subordonates: int):
        self.number_of_subordonates = number_of_subordonates
        super().__init__(name, base_salary)


def salary_employee(employee: Employee) -> float:
    return employee.base_salary


def salary_manager(manager: Manager) -> float:
    return manager.base_salary + 100 * manager.number_of_subordonates

From a tuple of managers or salaries, I want to be able to compute the total cost of their salaries for my company using a Callable[[Employee], float] like salary_employee and same thing for managers :

In [2]:
def sum_all_manager_salary(
    tup: tuple[Manager], salary: Callable[[Manager], float]
) -> float:
    return sum(salary(x) for x in tup)


def sum_all_employee_salary(
    tup: tuple[Employee], salary: Callable[[Employee], float]
) -> float:
    return sum(salary(x) for x in tup)

Now I define a couple of managers and employees and use them to define two functions to compute the total cost of these managers and employees with functions to compute individual salaries for employees and managers to be given as arguments (Here I should define 10 employee but for this example, I just define two of them) :

In [3]:
tup_manager = (Manager("Bob", 3000, 7), Manager("Alice", 4000, 3))
tup_employee = (Employee("Mike", 2000), Employee("Ed", 1000))
get_sum_all_manager_salary = partial(sum_all_manager_salary, tup_manager)
get_sum_all_employee_salary = partial(sum_all_employee_salary, tup_employee)

Let's try now to execute these functions to find the cost for my company of my managers and employees :

In [4]:
print(
    "Total salary of all my managers :", get_sum_all_manager_salary(salary=salary_manager)
)

print(
    "Total salary of all my employees not managers :",
    get_sum_all_employee_salary(salary=salary_employee),
)

Total salary of all my managers : 8000
Total salary of all my employees not managers : 3000


Can I use the function salary_employee with get_sum_all_manager_salary ?

In [5]:
print(
    "Total salary of all my managers (but computed as employees not managers) :",
    get_sum_all_manager_salary(salary=salary_employee),
)

Total salary of all my managers (but computed as employees not managers) : 7000


It works ! Of course the managers won't be happy to lose their bonus proportional to the number of subordonates as their salary is now computed with salary_employee ; but it works.

Can I use the function salary_manager with get_sum_all_salary_salary ?

In [6]:
print(
    "Total salary of all my salaries (but computed as managers) :",
    get_sum_all_employee_salary(salary=salary_manager),
)

AttributeError: 'Employee' object has no attribute 'number_of_subordonates'

Salary_manager compute the salary of a manager using number_of_subordonates which don't exist for an employee not manager so it can't work...

Well, I can use salary_employee instead of salary_manager but (in general) not salary_manager instead of salary_employee so callable[[Employee], float] is a (strict) subtype of callable[[Manager], float].

Another way to understand this is that (here) I have only one way to compute the salary of an employee (not manager) base_salary but many ways to compute the salary of a manager, for example :

base_salary + 100 * number_of_subordonates
base_salary + 200 * number_of_subordonates
base_salary + 200 * log(number_of_subordonates)
base_salary
The set of Callable[[Manager], float] (for the computing of a salary) is enormous while the set of Callable[[Employee], float] is a singleton (from a mathematical perspective for the computing of a salary) included in the set of Callable[[Manager], float].